In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("hive-queries") \
    .master("spark://spark-master:7077")\
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

# spark.stop()

In [3]:
#Use WES Database
spark.sql("USE wes")
current_db = spark.sql("SELECT current_database()").collect()[0][0]
print(current_db)

wes


In [4]:
from pyspark.sql import SparkSession

def run_spark_sql(sql_file_path, save_file=None):
    """
    Executes a SQL query from a file in Spark SQL.

    Parameters:
    sql_file_path (str): Path to the SQL query file.
    save_file (str, optional): Path to save the output. If None, the output is not saved.
    """

    # Start Spark Session
    spark = SparkSession.builder \
    .appName("hive-queries") \
    .master("spark://spark-master:7077")\
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()
    
    # Use WES
    spark.sql("USE wes")

    # Read SQL query from the file
    with open("sql/"+sql_file_path, 'r') as file:
        sql_query = file.read()

    # Execute the SQL query
    result = spark.sql(sql_query)

    # Show the first 20 rows
    result.show()

    # Save the result to a file if a save path is provided
    if save_file:
        result.toPandas().to_csv("output/"+save_file,  index=False)

    # Stop the Spark Session
    spark.stop()



In [41]:
run_spark_sql("ren_vs_fossil.sql", "insight-1") 

+----+----------------------+-----------------------+
|year|renewables_consumption|fossil_fuel_consumption|
+----+----------------------+-----------------------+
|1990|              6863.432|               83064.32|
|1991|              7032.226|              83294.062|
|1992|              7059.343|              83769.078|
|1993|              7474.867|              83915.758|
|1994|              7548.948|              84960.594|
|1995|              7956.042|              86531.617|
|1996|              8064.391|              89078.883|
|1997|              8241.819|              90023.977|
|1998|              8335.938|              90476.656|
|1999|              8437.115|                91940.5|
|2000|              8626.927|              94406.922|
|2001|              8417.771|                95559.0|
|2002|              8622.095|              97669.164|
|2003|               8650.36|             101887.992|
|2004|              9311.386|             106794.469|
|2005|              9684.451

NameError: name 'save_path' is not defined

In [6]:
run_spark_sql("energy_share.sql", "insight-2.csv") 

+-------+----+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+----------------------+
|country|year|     biofuel_share|        coal_share|         gas_share|         oil_share|     nuclear_share|       hydro_share|         solar_share|          wind_share|other_renewables_share|
+-------+----+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+----------------------+
| Brazil|2022| 6.495553805755959| 4.366916330793739| 8.584512054242953|  37.3355211495915|0.9778593674787666|29.894807095332972|   2.108636782320784|   5.713593839894054|     4.522599574589296|
| Brazil|2021| 6.819056367572472| 5.542601248505183|11.327425945270777|37.181873429916514|1.0340636351160988|26.588259346285152|  1.2276407508954903|   5.297297145913793|    4.9817821305245245|
| Brazil|2020| 7.2544579727360